# 🏴‍☠️ AN2DL25 Challenge 1 — Pirate Pain Classification

This notebook implements a full deep-learning pipeline for multivariate time-series classification of the Pirate Pain dataset. It is inspired by the Lecture 4 notebook (`Timeseries Classification (1).ipynb`) but adapted for the competition setting, including data preparation, model training (RNN/GRU/LSTM variants), evaluation, and test-time inference.



In [ ]:
# %%capture
# !pip install -q -r requirements.txt




In [ ]:
import os
import random
import math
from pathlib import Path
from typing import Tuple, Dict, Optional

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader



In [ ]:
SEED = 42
DATA_DIR = Path('/Users/md101ta/Desktop/Pirates/data')
OUTPUT_DIR = Path('/Users/md101ta/Desktop/Pirates/outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

# Reproducibility
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    DEVICE = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    DEVICE = torch.device('cpu')

print(f'Device: {DEVICE}')


In [ ]:
def load_data(data_dir: Path) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    X_train = pd.read_csv(data_dir / 'pirate_pain_train.csv')
    y_train = pd.read_csv(data_dir / 'pirate_pain_train_labels.csv')
    X_test = pd.read_csv(data_dir / 'pirate_pain_test.csv')
    return X_train, y_train, X_test


X_train_raw, y_train, X_test_raw = load_data(DATA_DIR)
print(X_train_raw.shape, y_train.shape, X_test_raw.shape)



In [ ]:
FEATURE_COLUMNS = [col for col in X_train_raw.columns if col not in ['sample_index', 'time']]
TIME_STEPS = X_train_raw['time'].nunique()
NUM_FEATURES = len(FEATURE_COLUMNS)
NUM_CLASSES = y_train['label'].nunique()
print(f'Time steps: {TIME_STEPS} | Features: {NUM_FEATURES} | Classes: {NUM_CLASSES}')

y_train['label'].value_counts().plot(kind='bar', title='Class distribution')
plt.show()



In [ ]:
LABEL2IDX = {label: idx for idx, label in enumerate(sorted(y_train['label'].unique()))}
IDX2LABEL = {idx: label for label, idx in LABEL2IDX.items()}
print('Label mapping:', LABEL2IDX)


def pivot_timeseries(df: pd.DataFrame) -> np.ndarray:
    pivoted = (
        df.pivot(index='sample_index', columns='time', values=FEATURE_COLUMNS)
          .sort_index(axis=0)
          .sort_index(axis=1, level=1)
    )
    data = pivoted.to_numpy().reshape(-1, TIME_STEPS, NUM_FEATURES)
    return data


X_train_np = pivot_timeseries(X_train_raw)
X_test_np = pivot_timeseries(X_test_raw)
y_train_idx = y_train.set_index('sample_index').loc[pd.unique(X_train_raw['sample_index'])]['label'].map(LABEL2IDX).to_numpy()

print(X_train_np.shape, y_train_idx.shape, X_test_np.shape)



In [ ]:
def compute_normalization_stats(data: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    # data shape: (N, T, F)
    mean = data.reshape(-1, NUM_FEATURES).mean(axis=0)
    std = data.reshape(-1, NUM_FEATURES).std(axis=0) + 1e-6
    return mean, std


def normalize(data: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
    return (data - mean) / std


feat_mean, feat_std = compute_normalization_stats(X_train_np)
X_train_np = normalize(X_train_np, feat_mean, feat_std)
X_test_np = normalize(X_test_np, feat_mean, feat_std)



In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data: np.ndarray, labels: Optional[np.ndarray] = None):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = None if labels is None else torch.tensor(labels, dtype=torch.long)

    def __len__(self) -> int:
        return self.data.shape[0]

    def __getitem__(self, idx: int):
        if self.labels is None:
            return self.data[idx]
        return self.data[idx], self.labels[idx]



In [ ]:
def create_dataloaders(X: np.ndarray, y: np.ndarray, valid_size: float = 0.2, batch_size: int = 64):
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=valid_size, random_state=SEED, stratify=y
    )

    train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=batch_size, shuffle=True, drop_last=False)
    valid_loader = DataLoader(TimeSeriesDataset(X_valid, y_valid), batch_size=batch_size, shuffle=False, drop_last=False)
    return train_loader, valid_loader, (X_train, y_train, X_valid, y_valid)


BATCH_SIZE = 64
train_loader, valid_loader, (X_train_split, y_train_split, X_valid_split, y_valid_split) = create_dataloaders(X_train_np, y_train_idx, batch_size=BATCH_SIZE)



In [ ]:
class RecurrentBackbone(nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int = 128,
        num_layers: int = 2,
        dropout: float = 0.2,
        rnn_type: str = 'lstm',
        bidirectional: bool = True,
    ):
        super().__init__()
        rnn_cls = {
            'rnn': nn.RNN,
            'gru': nn.GRU,
            'lstm': nn.LSTM,
        }[rnn_type.lower()]
        self.rnn_type = rnn_type.lower()
        self.bidirectional = bidirectional
        self.rnn = rnn_cls(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
            bidirectional=bidirectional,
        )
        proj_input = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(proj_input, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, NUM_CLASSES),
        )

    def forward(self, x):
        out, _ = self.rnn(x)
        # use last time-step hidden state
        last = out[:, -1, :]
        return self.head(last)



In [ ]:
def accuracy_from_logits(logits: torch.Tensor, targets: torch.Tensor) -> float:
    preds = torch.argmax(logits, dim=1)
    return (preds == targets).float().mean().item()


def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, total_acc = 0.0, 0.0
    for batch, (inputs, targets) in enumerate(loader):
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)
        optimizer.zero_grad()
        logits = model(inputs)
        loss = criterion(logits, targets)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        total_acc += accuracy_from_logits(logits.detach(), targets) * inputs.size(0)

    total_loss /= len(loader.dataset)
    total_acc /= len(loader.dataset)
    return total_loss, total_acc


def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_acc = 0.0, 0.0
    preds, targets_all = [], []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)
            logits = model(inputs)
            loss = criterion(logits, targets)

            total_loss += loss.item() * inputs.size(0)
            total_acc += accuracy_from_logits(logits, targets) * inputs.size(0)

            preds.append(torch.argmax(logits, dim=1).cpu())
            targets_all.append(targets.cpu())

    total_loss /= len(loader.dataset)
    total_acc /= len(loader.dataset)
    preds = torch.cat(preds)
    targets_all = torch.cat(targets_all)
    f1 = f1_score(targets_all, preds, average='macro')
    return total_loss, total_acc, f1, preds.numpy(), targets_all.numpy()



In [ ]:
def train_model(
    rnn_type: str = 'lstm',
    hidden_size: int = 128,
    num_layers: int = 2,
    dropout: float = 0.3,
    lr: float = 1e-3,
    weight_decay: float = 1e-4,
    epochs: int = 30,
    batch_size: int = 64,
):
    train_loader, valid_loader, _ = create_dataloaders(X_train_np, y_train_idx, batch_size=batch_size)

    model = RecurrentBackbone(
        input_size=NUM_FEATURES,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout,
        rnn_type=rnn_type,
    ).to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)

    history = {
        'train_loss': [],
        'train_acc': [],
        'valid_loss': [],
        'valid_acc': [],
        'valid_f1': [],
    }

    best_f1 = -np.inf
    best_state = None

    for epoch in range(1, epochs + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer)
        valid_loss, valid_acc, valid_f1, preds, targets = evaluate(model, valid_loader, criterion)
        scheduler.step(valid_f1)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['valid_loss'].append(valid_loss)
        history['valid_acc'].append(valid_acc)
        history['valid_f1'].append(valid_f1)

        if valid_f1 > best_f1:
            best_f1 = valid_f1
            best_state = {
                'model_state': model.state_dict(),
                'history': history.copy(),
                'epoch': epoch,
                'preds': preds,
                'targets': targets,
                'config': {
                    'rnn_type': rnn_type,
                    'hidden_size': hidden_size,
                    'num_layers': num_layers,
                    'dropout': dropout,
                    'lr': lr,
                    'weight_decay': weight_decay,
                    'epochs': epochs,
                    'batch_size': batch_size,
                }
            }
            torch.save(best_state, OUTPUT_DIR / f'best_{rnn_type}.pt')

        print(
            f"Epoch {epoch:02d} | Train Loss {train_loss:.4f} Acc {train_acc:.3f} | "
            f"Valid Loss {valid_loss:.4f} Acc {valid_acc:.3f} F1 {valid_f1:.3f}"
        )

    model.load_state_dict(best_state['model_state'])
    return model, history, best_state



In [ ]:
# Example: train baseline LSTM (adjust hyperparameters as needed)
lstm_model, lstm_history, lstm_state = train_model(
    rnn_type='lstm',
    hidden_size=192,
    num_layers=2,
    dropout=0.4,
    lr=2e-3,
    weight_decay=1e-4,
    epochs=30,
    batch_size=128,
)



In [ ]:
def plot_history(history: Dict[str, list]):
    epochs = range(1, len(history['train_loss']) + 1)
    fig, axs = plt.subplots(1, 3, figsize=(18, 4))

    axs[0].plot(epochs, history['train_loss'], label='Train')
    axs[0].plot(epochs, history['valid_loss'], label='Valid')
    axs[0].set_title('Loss')
    axs[0].set_xlabel('Epoch')
    axs[0].legend()

    axs[1].plot(epochs, history['train_acc'], label='Train')
    axs[1].plot(epochs, history['valid_acc'], label='Valid')
    axs[1].set_title('Accuracy')
    axs[1].set_xlabel('Epoch')
    axs[1].legend()

    axs[2].plot(epochs, history['valid_f1'], label='Valid F1', color='green')
    axs[2].set_title('Macro F1')
    axs[2].set_xlabel('Epoch')

    plt.tight_layout()
    plt.show()


plot_history(lstm_history)



In [ ]:
valid_loader_final = DataLoader(TimeSeriesDataset(X_valid_split, y_valid_split), batch_size=128, shuffle=False)
criterion = nn.CrossEntropyLoss()
_, _, f1_valid, preds_valid, targets_valid = evaluate(lstm_model, valid_loader_final, criterion)
print(f'Validation macro F1: {f1_valid:.3f}')
print(classification_report(targets_valid, preds_valid, target_names=[IDX2LABEL[i] for i in range(NUM_CLASSES)]))

cf = confusion_matrix(targets_valid, preds_valid)
plt.figure(figsize=(6, 5))
sns.heatmap(cf, annot=True, fmt='d', cmap='Blues', xticklabels=[IDX2LABEL[i] for i in range(NUM_CLASSES)], yticklabels=[IDX2LABEL[i] for i in range(NUM_CLASSES)])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Validation Confusion Matrix')
plt.show()



In [ ]:
test_loader = DataLoader(TimeSeriesDataset(X_test_np), batch_size=256, shuffle=False)

lstm_model.eval()
test_preds = []
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(DEVICE)
        logits = lstm_model(inputs)
        test_preds.append(torch.argmax(logits, dim=1).cpu().numpy())

test_preds = np.concatenate(test_preds)
test_labels = [IDX2LABEL[idx] for idx in test_preds]

submission = pd.DataFrame({
    'sample_index': pd.unique(X_test_raw['sample_index']),
    'label': test_labels,
})
submission.to_csv(OUTPUT_DIR / 'submission_lstm.csv', index=False)
submission.head()



## Next Steps

- Tune hyperparameters (hidden size, depth, dropout, learning rate) and evaluate on validation splits.
- Experiment with different recurrent cells by calling `train_model(rnn_type='gru')` or `train_model(rnn_type='rnn')`.
- Implement k-fold cross-validation by adapting the `create_dataloaders` helper to loop over `StratifiedKFold` splits.
- Add ensembling and/or temporal pooling strategies (attention, mean pooling across time) for further gains.
- Document results inside this notebook so it is fully reproducible for the final submission package.

